In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score
from sklearn.metrics import recall_score, f1_score, roc_auc_score
import keras.models as models
import keras.layers as layers
import keras.metrics as metrics

In [60]:
def get_clf_eval(y_test, pred):
    pred = (pred > 0.6)
    acc = accuracy_score(y_test, pred)
    pre = precision_score(y_test, pred, pos_label=1)
    re = recall_score(y_test, pred, pos_label=1)
    f1 = f1_score(y_test, pred, pos_label=1)
    auc = roc_auc_score(y_test, pred)

    return acc, pre, re, f1, auc

def print_clf_eval(y_test, pred):
    pred = (pred > 0.6)
    confusion = confusion_matrix(y_test, pred)
    acc, pre, re, f1, auc = get_clf_eval(y_test, pred)

    print("=Confusion matrix=")
    print(confusion)
    print("==================")

    print(f"Acc : {acc:.4f}, Pre : {pre:.4f}")
    print(f"Re : {re:.4f}, F1 : {f1:.4f}, AUC : {auc:.4f}")
    
def get_result(model, X_train, Y_train, X_test, Y_test):
    model.fit(X_train, Y_train)
    pred = (model.predict(X_test) > 0.6)
    
    return get_clf_eval(Y_test, pred)

In [3]:
data = pd.read_csv('Census_Income.csv')

In [4]:
data.dropna(axis=0, inplace=True)
data.reset_index(drop=True, inplace=True)
lb = LabelEncoder()
data['income'] = lb.fit_transform(data['income'])

In [5]:
scaler = StandardScaler()

data['age'] = scaler.fit_transform(np.array(data['age']).reshape(-1, 1))
data['fnlwgt'] = scaler.fit_transform(np.array(data['fnlwgt']).reshape(-1, 1))
data['education-num'] = scaler.fit_transform(np.array(data['education-num']).reshape(-1, 1))
data['capital-gain'] = scaler.fit_transform(np.array(data['capital-gain']).reshape(-1, 1))
data['capital-loss'] = scaler.fit_transform(np.array(data['capital-loss']).reshape(-1, 1))
data['hours-per-week'] = scaler.fit_transform(np.array(data['hours-per-week']).reshape(-1, 1))
data

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.037656,State-gov,-1.063678,Bachelors,1.138803,Never-married,Adm-clerical,Not-in-family,White,Male,0.154812,-0.231072,-0.094472,United-States,0
1,0.877496,Self-emp-not-inc,-1.009266,Bachelors,1.138803,Married-civ-spouse,Exec-managerial,Husband,White,Male,-0.146227,-0.231072,-2.421087,United-States,0
2,-0.038693,Private,0.233290,HS-grad,-0.445800,Divorced,Handlers-cleaners,Not-in-family,White,Male,-0.146227,-0.231072,-0.094472,United-States,0
3,1.106543,Private,0.412394,11th,-1.238102,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,-0.146227,-0.231072,-0.094472,United-States,0
4,-0.802184,Private,1.385969,Bachelors,1.138803,Married-civ-spouse,Prof-specialty,Wife,Black,Female,-0.146227,-0.231072,-0.094472,Cuba,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4926,0.343052,Private,-0.976976,Assoc-voc,0.346501,Married-civ-spouse,Craft-repair,Husband,White,Male,-0.146227,-0.231072,4.989614,United-States,0
4927,1.870034,Private,1.604011,Masters,1.534954,Separated,Prof-specialty,Not-in-family,White,Female,-0.146227,-0.231072,-0.094472,United-States,1
4928,0.572099,Private,-0.258878,7th-8th,-2.426555,Married-civ-spouse,Transport-moving,Husband,White,Male,-0.146227,-0.231072,-0.094472,United-States,0
4929,1.564637,Private,-0.105220,HS-grad,-0.445800,Married-civ-spouse,Sales,Husband,White,Male,-0.146227,-0.231072,-0.094472,United-States,0


In [6]:
X = data.iloc[:, 0:-1]
Y = data.iloc[:, -1]

In [7]:
X_log = X.copy()
Y_log = Y.copy()

for column in X_log.columns:
    if X_log[column].dtype == object:
        before = X_log[[column]]
        encoder = OneHotEncoder()
        temp = encoder.fit_transform(before)
        X_log.drop(column, axis=1, inplace=True)
        X_log = pd.concat([pd.DataFrame(temp.toarray(), columns=encoder.categories_), X_log], axis=1)
        del encoder
        
X_train_log, X_test_log, Y_train_log, Y_test_log = train_test_split(X_log, Y_log, test_size=0.3, random_state=13, stratify=Y)
X_train_log.reset_index(drop=True, inplace=True)
X_test_log.reset_index(drop=True, inplace=True)
Y_train_log.reset_index(drop=True, inplace=True)
Y_test_log.reset_index(drop=True, inplace=True)

In [16]:
X_rf = X.copy()
Y_rf = Y.copy()

for column in X_rf.columns:
    if X_rf[column].dtype == object:
        before = X_rf[[column]]
        encoder = LabelEncoder()
        temp = encoder.fit_transform(before)
        X_rf[column] = temp
        del encoder
        
X_train_rf, X_test_rf, Y_train_rf, Y_test_rf = train_test_split(X_rf, Y_rf, test_size=0.3, random_state=13, stratify=Y)
X_train_rf.reset_index(drop=True, inplace=True)
X_test_rf.reset_index(drop=True, inplace=True)
Y_train_rf.reset_index(drop=True, inplace=True)
Y_test_rf.reset_index(drop=True, inplace=True)

/Users/mintflavor/tensorflow_macos_venv/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [8]:
class_weight = {0:0.25, 1:0.75}

In [54]:
model_log = models.Sequential(name='Logistic')
model_log.add(layers.Dense(1, activation='sigmoid', input_shape=(X_train_log.shape[1],)))
#model.add(layers.Dense(1, activation='sigmoid'))
model_log.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [55]:
model_log.summary()

Model: "Logistic"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 1)                 103       
Total params: 103
Trainable params: 103
Non-trainable params: 0
_________________________________________________________________


In [56]:
model_log.fit(X_train_log, Y_train_log, epochs=200, use_multiprocessing=True, workers=-1)

Epoch 1/200
108/108 [==============================] - 0s 314us/step - loss: 0.6213 - accuracy: 0.7184
Epoch 2/200
108/108 [==============================] - 0s 272us/step - loss: 0.5203 - accuracy: 0.7633
Epoch 3/200
108/108 [==============================] - 0s 275us/step - loss: 0.4599 - accuracy: 0.7903
Epoch 4/200
108/108 [==============================] - 0s 267us/step - loss: 0.4429 - accuracy: 0.7861
Epoch 5/200
108/108 [==============================] - 0s 270us/step - loss: 0.4079 - accuracy: 0.8098
Epoch 6/200
108/108 [==============================] - 0s 273us/step - loss: 0.4018 - accuracy: 0.8103
Epoch 7/200
108/108 [==============================] - 0s 267us/step - loss: 0.3807 - accuracy: 0.8250
Epoch 8/200
108/108 [==============================] - 0s 264us/step - loss: 0.3904 - accuracy: 0.8158
Epoch 9/200
108/108 [==============================] - 0s 263us/step - loss: 0.3806 - accuracy: 0.8238
Epoch 10/200
108/108 [==============================] - 0s 275us/step - l

108/108 [==============================] - 0s 271us/step - loss: 0.3241 - accuracy: 0.8401
Epoch 80/200
108/108 [==============================] - 0s 268us/step - loss: 0.3241 - accuracy: 0.8414
Epoch 81/200
108/108 [==============================] - 0s 261us/step - loss: 0.3409 - accuracy: 0.8347
Epoch 82/200
108/108 [==============================] - 0s 264us/step - loss: 0.3359 - accuracy: 0.8368
Epoch 83/200
108/108 [==============================] - 0s 263us/step - loss: 0.3271 - accuracy: 0.8408
Epoch 84/200
108/108 [==============================] - 0s 263us/step - loss: 0.3253 - accuracy: 0.8434
Epoch 85/200
108/108 [==============================] - 0s 263us/step - loss: 0.3469 - accuracy: 0.8351
Epoch 86/200
108/108 [==============================] - 0s 267us/step - loss: 0.3378 - accuracy: 0.8311
Epoch 87/200
108/108 [==============================] - 0s 275us/step - loss: 0.3375 - accuracy: 0.8361
Epoch 88/200
108/108 [==============================] - 0s 272us/step - loss:

108/108 [==============================] - 0s 263us/step - loss: 0.3096 - accuracy: 0.8533
Epoch 158/200
108/108 [==============================] - 0s 262us/step - loss: 0.3221 - accuracy: 0.8474
Epoch 159/200
108/108 [==============================] - 0s 260us/step - loss: 0.3269 - accuracy: 0.8396
Epoch 160/200
108/108 [==============================] - 0s 260us/step - loss: 0.3384 - accuracy: 0.8424
Epoch 161/200
108/108 [==============================] - 0s 264us/step - loss: 0.3137 - accuracy: 0.8482
Epoch 162/200
108/108 [==============================] - 0s 258us/step - loss: 0.3290 - accuracy: 0.8453
Epoch 163/200
108/108 [==============================] - 0s 260us/step - loss: 0.3277 - accuracy: 0.8383
Epoch 164/200
108/108 [==============================] - 0s 257us/step - loss: 0.3411 - accuracy: 0.8382
Epoch 165/200
108/108 [==============================] - 0s 260us/step - loss: 0.3403 - accuracy: 0.8368
Epoch 166/200
108/108 [==============================] - 0s 260us/ste

In [57]:
model_log.evaluate(X_test_log, Y_test_log)

47/47 [==============================] - 0s 261us/step - loss: 0.3210 - accuracy: 0.8595


[0.3209892213344574, 0.8594594597816467]

In [62]:
temp = []
temp.append(get_result(model_log, X_train_log, Y_train_log, X_test_log, Y_test_log))

columns = ['Accuracy', 'Precision', 'Recall', 'F1-score', 'ROC-AUC']

pd.DataFrame(temp, index=["Log"], columns=columns)

108/108 [==============================] - 0s 317us/step - loss: 0.3275 - accuracy: 0.8418


,Accuracy,Precision,Recall,F1-score,ROC-AUC
Log,0.85,0.840183,0.495957,0.623729,0.732198


In [63]:
print_clf_eval(Y_test_log, model_log.predict(X_test_log))

=Confusion matrix=
[[1074   35]
 [ 187  184]]
Acc : 0.8500, Pre : 0.8402
Re : 0.4960, F1 : 0.6237, AUC : 0.7322


In [64]:
model_dnn = models.Sequential(name = 'DNN')
model_dnn.add(layers.Dense(1, activation='sigmoid', input_shape=(X_train_rf.shape[1],)))
model_dnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [65]:
model_dnn.summary()

Model: "DNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 1)                 15        
Total params: 15
Trainable params: 15
Non-trainable params: 0
_________________________________________________________________


In [66]:
model_dnn.fit(X_train_rf, Y_train_rf, epochs=200, use_multiprocessing=True, workers=-1)

Epoch 1/200
108/108 [==============================] - 0s 339us/step - loss: 1.8441 - accuracy: 0.7437
Epoch 2/200
108/108 [==============================] - 0s 277us/step - loss: 0.9850 - accuracy: 0.7111
Epoch 3/200
108/108 [==============================] - 0s 269us/step - loss: 0.9056 - accuracy: 0.7008
Epoch 4/200
108/108 [==============================] - 0s 268us/step - loss: 0.8080 - accuracy: 0.7086
Epoch 5/200
108/108 [==============================] - 0s 269us/step - loss: 0.7083 - accuracy: 0.7195
Epoch 6/200
108/108 [==============================] - 0s 266us/step - loss: 0.6546 - accuracy: 0.7171
Epoch 7/200
108/108 [==============================] - 0s 268us/step - loss: 0.6005 - accuracy: 0.7334
Epoch 8/200
108/108 [==============================] - 0s 265us/step - loss: 0.5696 - accuracy: 0.7333
Epoch 9/200
108/108 [==============================] - 0s 268us/step - loss: 0.5126 - accuracy: 0.7481
Epoch 10/200
108/108 [==============================] - 0s 267us/step - l

108/108 [==============================] - 0s 264us/step - loss: 0.4193 - accuracy: 0.8048
Epoch 80/200
108/108 [==============================] - 0s 270us/step - loss: 0.4053 - accuracy: 0.8111
Epoch 81/200
108/108 [==============================] - 0s 265us/step - loss: 0.4089 - accuracy: 0.8159
Epoch 82/200
108/108 [==============================] - 0s 265us/step - loss: 0.4058 - accuracy: 0.8089
Epoch 83/200
108/108 [==============================] - 0s 265us/step - loss: 0.4011 - accuracy: 0.8201
Epoch 84/200
108/108 [==============================] - 0s 258us/step - loss: 0.4061 - accuracy: 0.8128
Epoch 85/200
108/108 [==============================] - 0s 259us/step - loss: 0.4134 - accuracy: 0.8082
Epoch 86/200
108/108 [==============================] - 0s 258us/step - loss: 0.4215 - accuracy: 0.8021
Epoch 87/200
108/108 [==============================] - 0s 260us/step - loss: 0.4133 - accuracy: 0.8029
Epoch 88/200
108/108 [==============================] - 0s 260us/step - loss:

108/108 [==============================] - 0s 262us/step - loss: 0.4117 - accuracy: 0.8128
Epoch 158/200
108/108 [==============================] - 0s 262us/step - loss: 0.4242 - accuracy: 0.8060
Epoch 159/200
108/108 [==============================] - 0s 262us/step - loss: 0.4065 - accuracy: 0.8153
Epoch 160/200
108/108 [==============================] - 0s 262us/step - loss: 0.4194 - accuracy: 0.8058
Epoch 161/200
108/108 [==============================] - 0s 260us/step - loss: 0.4065 - accuracy: 0.8100
Epoch 162/200
108/108 [==============================] - 0s 257us/step - loss: 0.4049 - accuracy: 0.8056
Epoch 163/200
108/108 [==============================] - 0s 257us/step - loss: 0.4266 - accuracy: 0.8019
Epoch 164/200
108/108 [==============================] - 0s 265us/step - loss: 0.4053 - accuracy: 0.8135
Epoch 165/200
108/108 [==============================] - 0s 261us/step - loss: 0.4105 - accuracy: 0.8102
Epoch 166/200
108/108 [==============================] - 0s 267us/ste

In [67]:
model_dnn.evaluate(X_test_rf, Y_test_rf)

47/47 [==============================] - 0s 288us/step - loss: 0.4020 - accuracy: 0.8196


[0.40201324224472046, 0.8195946216583252]

In [68]:
temp = []
temp.append(get_result(model_dnn, X_train_rf, Y_train_rf, X_test_rf, Y_test_rf))

columns = ['Accuracy', 'Precision', 'Recall', 'F1-score', 'ROC-AUC']

pd.DataFrame(temp, index=["DNN"], columns=columns)

108/108 [==============================] - 0s 282us/step - loss: 0.4094 - accuracy: 0.8108


,Accuracy,Precision,Recall,F1-score,ROC-AUC
DNN,0.810135,0.826087,0.307278,0.447937,0.642818


In [69]:
print_clf_eval(Y_test_rf, model_dnn.predict(X_test_rf))

=Confusion matrix=
[[1085   24]
 [ 257  114]]
Acc : 0.8101, Pre : 0.8261
Re : 0.3073, F1 : 0.4479, AUC : 0.6428
